In [1]:
!pip3 install networkx

# textrank 사용 문장요약
## 1st textrank_korean

* window : 문맥으로 사용할 단어의 개수. 기본값 5로 주면 특정 단어의 좌우 5개씩, 총 10개 단어를 문맥으로 사용합니다.

* coef : 동시출현 빈도를 weight에 반영하는 비율입니다. 기본값은 1.0로, 동시출현 빈도를 weight에 전부 반영합니다. 0.0일 경우 빈도를 반영하지 않고 모든 간선의 weight을 1로 동일하게 간주합니다.

* threshold: 문서 요약시 관련있는 문장으로 여길 최소 유사도값. 기본값은 0.005이고, 이 값보다 작은 유사도를 가지는 문장쌍은 관련없는문장으로 처리합니다.

In [8]:
import networkx
import re
 
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
 
 
    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 
    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
 
        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)
 
        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)
 
        return both
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))


##출처: https://bab2min.tistory.com/570 [나의 큰 O는 logx야..]

In [9]:
tr = TextRank()
print('Load...')
from konlpy.tag import Komoran
tagger = Komoran()
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
tr.loadSents(RawSentenceReader('wiki/chosun.txt'), lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))
print('Build...')
tr.build()
ranks = tr.rank()
for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
print(tr.summarize(0.2))

Load...
Build...
0	0.011207935010104895	조선(朝鮮, 중세 한국어:
86	0.009366288421498281	 이 두 정책은 조선의 근본정책으로서 계속 계승되었다.
332	0.009113452185489009	 그런데 일본도 조선을 한국이라 부른 면도 있다.
284	0.00873013761167629	 일본은 이러한 무력을 배경으로 조선에게 개항을 강요하였다.
1	0.008387459623360849	 리조조선, 리씨조선, 약칭:
272	0.007881835827510236	이로부터 5년 뒤, 이번에는 미국이 조선을 침략하였다.
84	0.007478918075117014	 조선 국왕을 정식 국왕으로 승인한 것은 1401년(태종 1년) 태종 때였다.
34	0.007200455936494347	 또한 무능한 개혁 정책은 조선을 근본적인 근대화로 이끌지 못했다.
331	0.006486347111230253	 하지만 일본 등의 나라는 조선의 황제국 선포를 인정하지 않고 조선이라 불렀다.
204	0.006129112936612273	이 부분의 본문은 붕당입니다.
51	0.006110011273139255	 성종(재위 1469~94년)은 개국 이후의 문물제도를 정비하였다.
269	0.006093657593393048	 프랑스는 조선에 대해 사과와 손해 배상, 그리고 통상을 요구하였다.
12	0.006065652893776164	 성종은 개국 이후의 문물 제도를 정비했다.
154	0.0060157905387051	1592년, 일본을 통일한 도요토미 히데요시(풍신수길)는 20만 병력의 일본군을 조선에 대거 보내어 조선을 침략했다.
40	0.006012197653121029	 흔히 조선(朝鮮) 또는 조선 왕조(朝鮮王朝), 이씨 조선(李氏朝鮮), 이왕조(李王朝), 이조(李朝)라고 부르기도 하였다.
107	0.005962680472387903	15세기 말부터 지방의 사림 세력이 정계에서 세력을 키우기 시작했다.
13	0.005962680472387

In [10]:
tr = TextRank()
print('Load...')
from konlpy.tag import Komoran
tagger = Komoran()
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
tr.loadSents(RawSentenceReader('wiki_en/chosun.txt'), lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))
print('Build...')
tr.build()
ranks = tr.rank()
for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
print(tr.summarize(0.2))

Load...
Build...
0	0.09999999999999999	 사군육진 hanja:
1	0.09999999999999999	Direct communication between the king and the common people was possible through the sangeon (상언; 上言) written petition system and the gyeokjaeng (격쟁; 擊錚) oral petition system.
2	0.09999999999999999	The government officials were ranked in 18 levels, ranging from first senior rank (정1품, 正一品) down to ninth junior rank (종9품, 從九品) based on seniority and promotion, which was achieved through the royal decree based on examination or recommendation.
3	0.09999999999999999	The officials of 1st senior rank, 1st junior rank, and 2nd senior rank were addressed with honorific "dae-gam" (대감, 大監) while those of 2nd junior rank and 3rd senior rank were addressed with honorific "yeong-gam" (영감, 令監).
4	0.09999999999999999	 The Chief State Councillor (Yeonguijeong, 영의정, 領議政), Left State Councillor (Jwauijeong, 좌의정, 左議政), and Right State Councillor (Uuijeong, 우의정, 右議政) were the highest-ranking officials in the government (All three w

In [11]:
tr = TextRank(window=5, coef=1)
print('Load...')
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV'), ('없', 'VV') ])
tr.load(RawTaggerReader('wiki/chosun.txt'), lambda w: w not in stopword and (w[1] in ('NNG', 'NNP', 'VV', 'VA')))
print('Build...')
tr.build()
kw = tr.extract(0.1)
for k in sorted(kw, key=kw.get, reverse=True):
    print("%s\t%g" % (k, kw[k]))

Load...


java.lang.VirtualMachineErrorPyRaisable: java.lang.OutOfMemoryError: GC overhead limit exceeded

In [12]:
tr = TextRank(window=5, coef=1)
print('Load...')
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV'), ('없', 'VV') ])
tr.load(RawTaggerReader('wiki_en/chosun.txt'), lambda w: w not in stopword and (w[1] in ('NNG', 'NNP', 'VV', 'VA')))
print('Build...')
tr.build()
kw = tr.extract(0.1)
for k in sorted(kw, key=kw.get, reverse=True):
    print("%s\t%g" % (k, kw[k]))

Load...


java.lang.VirtualMachineErrorPyRaisable: java.lang.OutOfMemoryError: GC overhead limit exceeded

In [7]:
!pip3 install textrankr


  Running setup.py bdist_wheel for textrankr ... done
  Stored in directory: /Users/sunu/Library/Caches/pip/wheels/62/ea/be/6d156db74ee32e50a2bad7a6580a18b37ff4ce94dc5a54151e
Successfully built textrankr


## 2nd textrank_korean ...3줄요약

In [4]:
from __future__ import print_function
from textrankr import TextRank

f = open("wiki/chosun.txt", 'r')
data = f.read()
textrank = TextRank(data)
print(textrank.summarize())

/anaconda3/lib/python3.6/site-packages/networkx/generators/stochastic.py:56: UserWarning: zero out-degree for node 161
  warnings.warn('zero out-degree for node %s' % u)


조선 후기의 정치는 붕당을 중심으로 형성되었다.
조선 후기의 정치는 붕당을 중심으로 형성되었는데 마침내 서인은 17세기 중반의 예송논쟁에서 남인에게 권력을 넘겨준다.
1890년대에는 농민 수탈에 대한 저항으로 동학농민운동이 일어났고, 동학농민운동의 진압을 명분으로 조선에 들어온 청나라와 일본의 군대가 충돌하였고, 친일적인 갑오개혁이 있었다.


In [13]:
from __future__ import print_function
from textrankr import TextRank

f = open("wiki_en/chosun.txt", 'r')
data = f.read()
textrank = TextRank(data)
print(textrank.summarize())

/anaconda3/lib/python3.6/site-packages/networkx/generators/stochastic.py:56: UserWarning: zero out-degree for node 0
  warnings.warn('zero out-degree for node %s' % u)
/anaconda3/lib/python3.6/site-packages/networkx/generators/stochastic.py:56: UserWarning: zero out-degree for node 2
  warnings.warn('zero out-degree for node %s' % u)
/anaconda3/lib/python3.6/site-packages/networkx/generators/stochastic.py:56: UserWarning: zero out-degree for node 3
  warnings.warn('zero out-degree for node %s' % u)
/anaconda3/lib/python3.6/site-packages/networkx/generators/stochastic.py:56: UserWarning: zero out-degree for node 4
  warnings.warn('zero out-degree for node %s' % u)
/anaconda3/lib/python3.6/site-packages/networkx/generators/stochastic.py:56: UserWarning: zero out-degree for node 5
  warnings.warn('zero out-degree for node %s' % u)
/anaconda3/lib/python3.6/site-packages/networkx/generators/stochastic.py:56: UserWarning: zero out-degree for node 6
  warnings.warn('zero out-degree for node %

The officials from 1st senior rank to 3rd senior rank wore red robes while those from 3rd junior rank to 6th junior rank wore blue and those below wore green robes.[37].
It was headed by Chief Scholar (Daejehak·대제학), a part-time post of 2nd senior rank that served concurrently in another high post (such as in State Council), and Deputy Chief Scholar (Bujehak·부제학), a full-time post of 3rd senior rank that actually ran the office.
There were six royal secretaries (승지), one for each ministry, and all were of 3rd senior rank.


## 3rd textrank_english

In [17]:
!pip3 install summa


    100% |████████████████████████████████| 51kB 635kB/s ta 0:00:011
  Running setup.py bdist_wheel for summa ... done
  Stored in directory: /Users/sunu/Library/Caches/pip/wheels/56/6e/9f/7be3e8cbc8dfa18c1db9eb2ee85e9ee1dcc91e38c4d4e8c969
Successfully built summa


In [22]:
from summa.summarizer import summarize
f = open("wiki_en/chosun.txt", 'r')
data = f.read()

summarize(data, ratio=0.2)

f = open("wiki_en/chosun_min.txt", "w")
f.write(summary)

'The Joseon dynasty (also transcribed as Chosŏn or Chosun, Korean: 조선; officially the Kingdom of Great Joseon, Korean: 대조선국) was a Korean dynastic kingdom that lasted for approximately five centuries.\nJoseon was the last dynasty of Korea and its longest-ruling Confucian dynasty.\nJoseon consolidated its effective rule over the territory of current Korea and saw the height of classical Korean culture, trade, literature, and science and technology.\nHowever, the dynasty was severely weakened during the late 16th and early 17th centuries, when the Japanese invasions of Korea (1592–98) and the first and second Manchu invasions of 1636 nearly overran the Korean Peninsula, leading to an increasingly harsh isolationist policy, for which the country became known as the "hermit kingdom" in Western literature.\nHowever, whatever power the kingdom recovered during its isolation further waned as the 18th century came to a close, and faced with internal strife, power struggles, international press

# 요약된 문장기반 질문생성_한글

In [16]:
!python3 quest_ko.py wiki/chosun_min.txt


-----------INPUT TEXT-------------

조선(朝鮮, 중세 한국어:  리조조선, 리씨조선, 약칭: [5] 1393년 2월 15일에는 국명을 “조선(朝鮮)”으로 정하였고[6], 1394년에는 한양을 도읍으로 하여 여러 개혁을 단행했다.  성종은 개국 이후의 문물 제도를 정비했다.  15세기 말부터 지방의 사림 세력이 정계에서 세력을 키우기 시작했다.  선조 때에는 마침내 훈구파들이 정계에서 물러나고 사림들이 정계를 장악하게 되었다.  이후 사림들은 '붕당'을 형성하여 조선 정치의 꽃이자 역모 반란죄에 해당될 수 있는 붕당정치를 시행한다. 조선 후기의 정치는 붕당을 중심으로 형성되었다. 1680년의 경신환국으로 서인이 권력을 잡은 뒤 균형이 무너져, 서인은 남인을 철저히 탄압하였다.  그리고 풍양 조씨의 가문의 대표였던 조대비)가 흥선대원군과 결탁하여 흥선대원군의 둘째아들 명복(고종)을 조선 국왕 자리에 올린다. 고종의 아버지인 흥선대원군은 막강한 권력을 행사했다.  또한 무능한 개혁 정책은 조선을 근본적인 근대화로 이끌지 못했다. 1890년대에는 농민 수탈에 대한 저항으로 동학농민운동이 일어났고, 동학농민운동의 진압을 명분으로 조선에 들어온 청나라와 일본의 군대가 충돌하기도 했다.  흔히 조선(朝鮮) 또는 조선 왕조(朝鮮王朝), 이씨 조선(李氏朝鮮), 이왕조(李王朝), 이조(李朝)라고 부르기도 하였다. 조선(1392~1897)의 역사는 크게 전기와 후기로 나누며, 전기를 다시 둘로 나눠 전기와 중기로 보아 전기-중기-후기로 보기도 한다. 1393년에는 국명을 명으로부터 선택받아 조선으로 고치고 1394년에는 개경의 민심이 안 좋아 한양으로 천도하여 여러 개혁을 단행했다.  태종 이방원(재위 1400~18년)은 왕권을 강화하고 임금 중심의 통치 체제를 정비하기 위해 관료 제도를 정비했다. 세종대왕(재위 1418~1450년)은 학문 · 군사 · 과학 · 문화 등 모든 면에서 큰 업적을 이룩하였고 정치는 안정되어갔다.  성종(재위 1469~9

# 요약된 문장기반 질문생성_영어

In [23]:
!python3 quest.py wiki_en/chosun_min.txt


-----------INPUT TEXT-------------

The Joseon dynasty (also transcribed as Chosŏn or Chosun, Korean: 조선; officially the Kingdom of Great Joseon, Korean: 대조선국) was a Korean dynastic kingdom that lasted for approximately five centuries.
Joseon was the last dynasty of Korea and its longest-ruling Confucian dynasty.
Joseon consolidated its effective rule over the territory of current Korea and saw the height of classical Korean culture, trade, literature, and science and technology.
However, the dynasty was severely weakened during the late 16th and early 17th centuries, when the Japanese invasions of Korea (1592–98) and the first and second Manchu invasions of 1636 nearly overran the Korean Peninsula, leading to an increasingly harsh isolationist policy, for which the country became known as the "hermit kingdom" in Western literature.
However, whatever power the kingdom recovered during its isolation further waned as the 18th century came to a close, and faced with internal strife, powe


 Question: What has Joseon?

 Question: What creation is was?

 Question: What The Joseon expressing?

 Question: What marks Joseon?

 Question: What is Samsa?

 Question: What is Office?


 질문 : 조선은 무엇입니까?

 질문 : 어떤 창조물이 있었습니까?

 질문 : 조선은 무엇을 표현 하는가?

 질문 : 조선은 무엇인가?

 질문 : Samsa 란 무엇입니까?

 질문 : Office 란 무엇입니까?